In this file, the LEAR (Lagged Error Auto-Regressive) model will be applied to forecast the price based on historical price data. This serves as a benchmark model to evaluate the performance of other more complex forecasting methods. The LEAR model will use the historical data to predict future prices and assess its effectiveness in capturing trends and fluctuations in the pricing data.

In [1]:
# Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from functions import plot_comparison, evaluate_model

In [2]:
df = pd.read_csv('../../Data/zra_sgp_dam.csv')

# Convert the 'Date' column to datetime if it's not already
df['Date'] = pd.to_datetime(df['Date'])
# Set the Date column as index
df.set_index('Date', inplace=True)

In [3]:
# Generate lag features
df['price_lag30d'] = df['Price (USD/MWh)'].shift(24*30)
df['price_lag30d'] = df['price_lag30d'].fillna(df['Price (USD/MWh)'])  # Fill with actual price for 30-day lag

df['price_lag60d'] = df['Price (USD/MWh)'].shift(24*60)
df['price_lag60d'] = df['price_lag60d'].fillna(df['Price (USD/MWh)'])  # Fill with actual price for 60-day lag

df['price_lag90d'] = df['Price (USD/MWh)'].shift(24*90)
df['price_lag90d'] = df['price_lag90d'].fillna(df['Price (USD/MWh)'])  # Fill with actual price for 90-day lag


In [4]:
import numpy as np
def encode_cyclic(df, col, max_val):
    """"
    Time features like Hour, Month, day_of_week are cyclical, not linear. 
    Without encoding them properly, the model will misunderstand their relationships.
    """
    df[col + '_sin'] = np.sin(2 * np.pi * df[col] / max_val)
    df[col + '_cos'] = np.cos(2 * np.pi * df[col] / max_val)
    return df

def preprocess(df):
    df = df.copy()
    
    # Cyclic encode time features
    df = encode_cyclic(df, 'Hour', 24)
    df = encode_cyclic(df, 'Month', 12)
    df = encode_cyclic(df, 'day_of_week', 7)
    
    # Drop unused or problematic columns
    df = df.drop(columns=['Hour', 'Month', 'day_of_week'])  # Keep cyclic versions instead
    
    # Fill/clean if needed
    df = df.fillna(method='ffill').dropna()
    
    return df

def evaluate(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    # Directional accuracy (compare sign of changes)
    true_diff = np.diff(y_true)
    pred_diff = np.diff(y_pred)
    dae = np.mean(np.sign(true_diff) == np.sign(pred_diff))

    lower_pct = np.mean(y_pred < y_true) * 100
    return mae, rmse, r2, dae, lower_pct

In [5]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Preprocess (make sure 'Date' becomes index)
df_clean = preprocess(df)

# Define target
target_col = 'Price (USD/MWh)'
features = df_clean.drop(columns=[target_col])
target = df_clean[target_col]

# Columns by type
minmax_cols = ['Tati- normalised output', 'E_Grid (Mw)', 'Revenues (USD)', 
               'Flow_chavuma', 'Level_kariba', 'Flow_nana']
standard_cols = ['Volatility_1 Day', 'Volatility_3 Days', 'Volatility_7 Days', 'Volatility_30 Days',
                 'roc_49h', 'momentum_49h']
no_scaling_cols = ['Hour_sin', 'Hour_cos', 'Month_sin', 'Month_cos',
                   'day_of_week_sin', 'day_of_week_cos']

# Initialize scalers
minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()
scaler_y = MinMaxScaler()

# Copy clean DataFrame
df_scaled = df_clean.copy()

# Apply scalers to appropriate columns
df_scaled[minmax_cols] = minmax_scaler.fit_transform(df_clean[minmax_cols])
df_scaled[standard_cols] = standard_scaler.fit_transform(df_clean[standard_cols])

# Target scaling (fit only on the column, keep shape)
df_scaled["target_scaled"] = scaler_y.fit_transform(df_clean[[target_col]])

# Optionally retain unscaled target for reference
df_scaled[target_col] = target


/scratch-local/krasmussen.12594015/ipykernel_3686790/424768561.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').dropna()


In [6]:
import time
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

# --- Preprocessing (assume preprocess function is defined) ---
df_clean = preprocess(df)  # make sure 'Date' is index in this step

# Create lag features upfront (assume hourly data, 24 per day)
lags_days = [7, 30, 60, 90]
for lag in lags_days:
    col_name = f'price_lag{lag}d'
    df_clean[col_name] = df_clean['Price (USD/MWh)'].shift(24*lag)
    df_clean[col_name].fillna(df_clean['Price (USD/MWh)'], inplace=True)  # fill initial NAs with current price


/scratch-local/krasmussen.12594015/ipykernel_3686790/424768561.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').dropna()
/scratch-local/krasmussen.12594015/ipykernel_3686790/2773569774.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean[col_name].fillna(df_clean['Price (USD/MWh)'], inplace=True)  # fill initial NAs with current price


In [13]:
# Define lag columns as list
lag_cols = [f'price_lag{lag}d' for lag in lags_days]

# Define base features = all columns except lag columns, target, and target_scaled
exclude_cols = lag_cols + [target_col, 'target_scaled']
base_feature_cols = [col for col in df_scaled.columns if col not in exclude_cols]

results = []
lags = []

# Loop over lookbacks
for lookback in lags_days:
    # print(f"\n--- Lookback: {lookback} days ---")
    lags.append(lookback)
    current_lag = f'price_lag{lookback}d'
    if current_lag not in df_scaled.columns:
        print(f"Missing lag column: {current_lag}")
        continue
    
    # Features = base features + current lag only
    feature_cols =  [current_lag] + base_feature_cols
    
    X = df_scaled[feature_cols]
    y = df_scaled['target_scaled']

    # Train-validation-test split (70-15-15), no shuffle
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)
    
    # Initialize Lasso
    model = Lasso(alpha=0.1)
    
    # Train model and time
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    # Predictions
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)
    
    # Evaluate on validation
    val_mae, val_rmse, val_r2, val_dae, val_lower = evaluate(y_val, y_val_pred)
    # Evaluate on test
    test_mae, test_rmse, test_r2, test_dae, test_lower = evaluate(y_test, y_test_pred)
    
    # Save results
    results.append({
        'Lookback (days)': lookback,
        'Training Time (s)': round(training_time, 4),
        
        'Val MAE': round(val_mae, 4),
        'Val RMSE': round(val_rmse, 4),
        'Val R2': round(val_r2, 4),
        'Val DAE': round(val_dae, 4),
        'Val Lower Predictions %': round(val_lower, 2),
        
        'Test MAE': round(test_mae, 4),
        'Test RMSE': round(test_rmse, 4),
        'Test R2': round(test_r2, 4),
        'Test DAE': round(test_dae, 4),
        'Test Lower Predictions %': round(test_lower, 2),
    })
    print(f"\n% Validation Results (Lookback: {lookback} days)")
    print(f" & {lookback} & {val_mae:.2f} & {val_dae:.2f} & {1:.2f} & {val_rmse:.2f} & {val_r2:.2f} & {val_lower:.2f} \\\\")

    # Print LaTeX table rows for Test results
    print(f"% Test Results (Lookback: {lookback} days)")
    print(f" & {lookback} & {test_mae:.2f} & {test_dae:.2f} & {1:.2f} & {test_rmse:.2f} & {test_r2:.2f} & {test_lower:.2f} \\\\")

    # Feature importances (non-zero Lasso coefficients)
    coefs = model.coef_
    nonzero_features = [(feature, coef) for feature, coef in zip(X_train.columns, coefs) if coef != 0]

    # # Print for in Overleaf
    # print(f"\nNon-zero coefficients (Lookback: {lookback} days):")
    # for feature, coef in sorted(nonzero_features, key=lambda x: abs(x[1]), reverse=True):
    #     print(f"{feature}: {coef:.4f}")

results_df = pd.DataFrame(results)


% Validation Results (Lookback: 7 days)
 & 7 & 0.06 & 0.67 & 1.00 & 0.10 & 0.75 & 41.72 \\
% Test Results (Lookback: 7 days)
 & 7 & 0.07 & 0.63 & 1.00 & 0.11 & 0.76 & 40.53 \\

Non-zero coefficients (Lookback: 7 days):
price_lag7d: 0.0013
price_lag1d: 0.0005
ema_22h: 0.0004
price_lag14d: 0.0004
ema_168h: -0.0000
GlobHor (W/m²): -0.0000
EArray (kW): 0.0000
E_Grid (kW): -0.0000

% Validation Results (Lookback: 30 days)
 & 30 & 0.07 & 0.64 & 1.00 & 0.11 & 0.73 & 42.60 \\
% Test Results (Lookback: 30 days)
 & 30 & 0.08 & 0.61 & 1.00 & 0.11 & 0.74 & 41.16 \\

Non-zero coefficients (Lookback: 30 days):
price_lag14d: 0.0014
price_lag1d: 0.0008
ema_22h: 0.0006
price_lag30d: -0.0001
ema_168h: -0.0001
GlobHor (W/m²): -0.0000
EArray (kW): 0.0000
E_Grid (kW): -0.0000

% Validation Results (Lookback: 60 days)
 & 60 & 0.07 & 0.65 & 1.00 & 0.11 & 0.73 & 41.87 \\
% Test Results (Lookback: 60 days)
 & 60 & 0.08 & 0.62 & 1.00 & 0.11 & 0.74 & 38.93 \\

Non-zero coefficients (Lookback: 60 days):
price_la

In [8]:
print("\nSummary Results:")
print(results_df)


Summary Results:
   Lookback (days)  Training Time (s)  Val MAE  Val RMSE  Val R2  Val DAE  \
0                7             0.1922   0.0609    0.1014  0.7527   0.6683   
1               30             0.2165   0.0677    0.1062  0.7288   0.6397   
2               60             0.2158   0.0674    0.1056  0.7318   0.6531   
3               90             0.2089   0.0676    0.1046  0.7371   0.6519   

   Val Lower Predictions %  Test MAE  Test RMSE  Test R2  Test DAE  \
0                    41.72    0.0702     0.1066   0.7566    0.6303   
1                    42.60    0.0761     0.1108   0.7371    0.6112   
2                    41.87    0.0761     0.1102   0.7398    0.6241   
3                    39.73    0.0753     0.1086   0.7475    0.6259   

   Test Lower Predictions %  
0                     40.53  
1                     41.16  
2                     38.93  
3                     36.60  



Non-zero coefficients (Lookback: 90 days):
price_lag14d: 0.0013
price_lag1d: 0.0008
ema_22h: 0.0007
price_lag90d: 0.0002
ema_168h: -0.0002
GlobHor (W/m²): -0.0000
EArray (kW): 0.0000
E_Grid (kW): -0.0000


In [ ]:
plot_comparison(y_test,y_test_pred,df, 'month')

In [ ]:
evaluate_model(y_val, y_val_pred, y_test, y_test_pred, X_val, X_test, plot_fig=None)